In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pprint import pprint
import autosklearn.classification
from tensorflow.keras.datasets import cifar10


2023-05-11 23:34:45.530350: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 23:34:45.623844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 23:34:45.623862: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 23:34:46.202483: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Data Loading

In [16]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Flatten the images
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))
# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0
# Convert labels to 1D array
y_train = y_train.ravel()
y_test = y_test.ravel()

## Build and fit a classifier

In [23]:

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=1000,
    per_run_time_limit=100,
    tmp_folder="/tmp/autosklearn_classification_example_tmp",
     n_jobs=-1
)
automl.fit(X_train, y_train, dataset_name="cifar10")

[WARNING] [2023-05-11 17:57:54,965:Client-AutoML(1):cifar10] /home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/autosklearn/util/data.py:464 UserWarning:Dataset too large for allocated memory 307.20000000000005MB, reduced the precision from float32 to <class 'numpy.float32'>
[WARNING] [2023-05-11 17:57:55,041:Client-AutoML(1):cifar10] /home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/autosklearn/util/data.py:490 UserWarning:Dataset too large for allocated memory 307.20000000000005MB, reduced number of samples from 50000 to 26214.
[WARNING] [2023-05-11 17:58:33,660:Client-EnsembleBuilder] No runs were available to build an ensemble from


AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      n_jobs=-1, per_run_time_limit=100,
                      time_left_for_this_task=1000,
                      tmp_folder='/tmp/autosklearn_classification_example_tmp')

## View the models found by auto-sklearn

In [20]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost   duration
model_id                                                 
149          1             0.02  lda  0.600277  16.079427
177          2             0.22  lda  0.600855  21.823135
162          3             0.12  lda  0.601549  14.877579
203          4             0.04  lda  0.602474  17.971540
156          5             0.12  lda  0.602589  14.541234
215          6             0.06  lda  0.602705  18.554503
153          7             0.04  lda  0.604554  23.647570
171          8             0.02  lda  0.604554  20.862972
145          9             0.04  lda  0.605942  23.822584
216         10             0.02  lda  0.606519  14.271788
196         11             0.02  lda  0.606751  23.061650
125         12             0.06  lda  0.607097  14.450578
158         13             0.02  lda  0.607444  12.832135
222         14             0.20  lda  0.608600  11.247762


## Print the final ensemble constructed by auto-sklearn

In [21]:
pprint(automl.show_models(), indent=4)

{   125: {   'balancing': Balancing(random_state=1, strategy='weighting'),
             'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f40e9bd0280>,
             'cost': 0.6070974453820368,
             'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f417b6bb850>,
             'ensemble_weight': 0.06,
             'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f4130923a00>,
             'model_id': 125,
             'rank': 1,
             'sklearn_classifier': LinearDiscriminantAnalysis(shrinkage=0.47991851861431317, solver='lsqr',
                           tol=0.0014306822597258818)},
    145: {   'balancing': Balancing(random_state=1, strategy='weighting'),
             'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f410ff0f070>,
             'cost': 0.60594150965

## Get the Score of the final ensemble

In [25]:
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions)) 

Accuracy score: 0.4645
